# Usage

In [1]:
from nlptext.base import BasicObject

########### Wiki ###########
CORPUSPath = 'corpus/wiki_sample/'

Corpus2GroupMethod = '.txt'

Group2TextMethod   = 'line'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
min_token_freq = 1

use_hyper = True

anno = False
anno_keywords = {}


BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = False, anno_keywords = anno_keywords)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/wiki_sample/sample_wiki_smp.txt


Loading model cost 0.730 seconds.
Prefix dict has been built succesfully.


Total Num of All    Tokens 13878
Total Num of Unique Tokens 1087
CORPUS	it is Dumped into file: data/wiki_sample/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/wiki_sample/char/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/wiki_sample/char/Pyramid/TEXT.p
TEXT	the length of it is   : 100
SENT	it is Dumped into file: data/wiki_sample/char/Pyramid/SENT.p
SENT	the length of it is   : 100
TOKEN	it is Dumped into file: data/wiki_sample/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 13878
**************************************** 

pos-bioes	is Dumped into file: data/wiki_sample/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/wiki_sample/char/Vocab/pos-bioes.tsv
pos-bioes	is Dumped into file: data/wiki_sample/char/Vocab/token.voc
pos-bioes	the length of it is   : 1087
		Write to: data/wiki_sample/char/Vocab/token.tsv
****************************************


## Boson

In [2]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/boson/'

Corpus2GroupMethod = '.txt'

Group2TextMethod   = 'line'

Text2SentMethod  = 're'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'anno_embed_in_text'
anno_keywords = {}

BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/boson/bosonNER.txt
Total Num of All    Tokens 533491
Total Num of Unique Tokens 3865
CORPUS	it is Dumped into file: data/boson/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/boson/char/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/boson/char/Pyramid/TEXT.p
TEXT	the length of it is   : 1961
SENT	it is Dumped into file: data/boson/char/Pyramid/SENT.p
SENT	the length of it is   : 10214
TOKEN	it is Dumped into file: data/boson/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 533491
**************************************** 

pos-bioes	is Dumped into file: data/boson/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/boson/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/boson/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 25
		Write to: data/boson/char/Vocab/annoE-bioes.tsv
annoE-bioes	is Dumped into file: data/boson/char/Vocab/token.voc
a

## ResumeCN

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/ResumeCN/'

Corpus2GroupMethod = '.bmes'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': '',
    'suffix': False,
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/ResumeCN/test.char.bmes
[['常', 0, 'B-NAME'], ['建', 1, 'M-NAME'], ['良', 2, 'E-NAME']]
常建良,男,
[]
[['工', 8, 'B-PRO'], ['科', 9, 'E-PRO'], ['学', 10, 'B-EDU'], ['士', 11, 'E-EDU'], ['高', 13, 'B-TITLE'], ['级', 14, 'M-TITLE'], ['工', 15, 'M-TITLE'], ['程', 16, 'M-TITLE'], ['师', 17, 'E-TITLE'], ['北', 19, 'B-ORG'], ['京', 20, 'M-ORG'], ['物', 21, 'M-ORG'], ['资', 22, 'M-ORG'], ['学', 23, 'M-ORG'], ['院', 24, 'E-ORG'], ['客', 25, 'B-TITLE'], ['座', 26, 'M-TITLE'], ['副', 27, 'M-TITLE'], ['教', 28, 'M-TITLE'], ['授', 29, 'E-TITLE']]
1963年出生,工科学士,高级工程师,北京物资学院客座副教授。
[]
[['国', 14, 'B-ORG'], ['家', 15, 'M-ORG'], ['物', 16, 'M-ORG'], ['资', 17, 'M-ORG'], ['局', 18, 'E-ORG'], ['物', 20, 'B-ORG'], ['资', 21, 'M-ORG'], ['部', 22, 'E-ORG'], ['国', 24, 'B-ORG'], ['内', 25, 'M-ORG'], ['贸', 26, 'M-ORG'], ['易', 27, 'M-ORG'], ['部', 28, 'M-ORG'], ['金', 29, 'M-ORG'], ['属', 30, 'M-ORG'], ['材', 31, 'M-ORG'], ['料', 32, 'M-ORG'], ['流', 33, 'M-ORG'], ['通', 34, 'M-ORG'], ['司', 35, 'E-ORG'], ['科', 64, 'B-TITLE'], ['员', 65, 'E-T

[['北', 2, 'B-ORG'], ['京', 3, 'M-ORG'], ['双', 4, 'M-ORG'], ['鹤', 5, 'M-ORG'], ['药', 6, 'M-ORG'], ['业', 7, 'M-ORG'], ['经', 8, 'M-ORG'], ['营', 9, 'M-ORG'], ['有', 10, 'M-ORG'], ['限', 11, 'M-ORG'], ['责', 12, 'M-ORG'], ['任', 13, 'M-ORG'], ['公', 14, 'M-ORG'], ['司', 15, 'E-ORG'], ['财', 16, 'B-TITLE'], ['务', 17, 'M-TITLE'], ['总', 18, 'M-TITLE'], ['监', 19, 'E-TITLE'], ['副', 21, 'B-TITLE'], ['总', 22, 'M-TITLE'], ['经', 23, 'M-TITLE'], ['理', 24, 'E-TITLE']]
曾任北京双鹤药业经营有限责任公司财务总监、副总经理;
[]
[['北', 0, 'B-ORG'], ['京', 1, 'M-ORG'], ['双', 2, 'M-ORG'], ['鹤', 3, 'M-ORG'], ['药', 4, 'M-ORG'], ['业', 5, 'M-ORG'], ['股', 6, 'M-ORG'], ['份', 7, 'M-ORG'], ['有', 8, 'M-ORG'], ['限', 9, 'M-ORG'], ['公', 10, 'M-ORG'], ['司', 11, 'E-ORG'], ['金', 12, 'B-TITLE'], ['鹤', 13, 'M-TITLE'], ['工', 14, 'M-TITLE'], ['程', 15, 'M-TITLE'], ['(', 16, 'M-TITLE'], ['信', 17, 'M-TITLE'], ['息', 18, 'M-TITLE'], ['化', 19, 'M-TITLE'], ['建', 20, 'M-TITLE'], ['设', 21, 'M-TITLE'], [')', 22, 'M-TITLE'], ['总', 23, 'M-TITLE'], ['经', 24, 'M-TITLE'], ['理'

[['易', 0, 'B-NAME'], ['睿', 1, 'E-NAME']]
易睿先生,
[]
[['中', 7, 'B-CONT'], ['国', 8, 'M-CONT'], ['籍', 9, 'E-CONT'], ['深', 11, 'B-ORG'], ['圳', 12, 'M-ORG'], ['大', 13, 'M-ORG'], ['学', 14, 'E-ORG'], ['电', 15, 'B-PRO'], ['子', 16, 'M-PRO'], ['与', 17, 'M-PRO'], ['通', 18, 'M-PRO'], ['讯', 19, 'M-PRO'], ['专', 20, 'M-PRO'], ['业', 21, 'E-PRO'], ['硕', 22, 'B-EDU'], ['士', 23, 'E-EDU']]
1978年生,中国籍,深圳大学电子与通讯专业硕士。
[]
[['深', 17, 'B-ORG'], ['圳', 18, 'M-ORG'], ['市', 19, 'M-ORG'], ['冠', 20, 'M-ORG'], ['日', 21, 'M-ORG'], ['科', 22, 'M-ORG'], ['技', 23, 'M-ORG'], ['有', 24, 'M-ORG'], ['限', 25, 'M-ORG'], ['公', 26, 'M-ORG'], ['司', 27, 'E-ORG'], ['部', 28, 'B-TITLE'], ['门', 29, 'M-TITLE'], ['经', 30, 'M-TITLE'], ['理', 31, 'E-TITLE']]
2001年4月至2005年11月任深圳市冠日科技有限公司部门经理;
[]
[['深', 7, 'B-ORG'], ['圳', 8, 'M-ORG'], ['市', 9, 'M-ORG'], ['同', 10, 'M-ORG'], ['洲', 11, 'M-ORG'], ['电', 12, 'M-ORG'], ['子', 13, 'M-ORG'], ['股', 14, 'M-ORG'], ['份', 15, 'M-ORG'], ['有', 16, 'M-ORG'], ['限', 17, 'M-ORG'], ['公', 18, 'M-ORG'], ['司', 19, 'E-ORG

[['北', 2, 'B-ORG'], ['方', 3, 'M-ORG'], ['财', 4, 'M-ORG'], ['务', 5, 'M-ORG'], ['咨', 6, 'M-ORG'], ['询', 7, 'M-ORG'], ['有', 8, 'M-ORG'], ['限', 9, 'M-ORG'], ['责', 10, 'M-ORG'], ['任', 11, 'M-ORG'], ['公', 12, 'M-ORG'], ['司', 13, 'E-ORG'], ['总', 14, 'B-TITLE'], ['经', 15, 'M-TITLE'], ['理', 16, 'E-TITLE'], ['中', 21, 'B-ORG'], ['纺', 22, 'M-ORG'], ['投', 23, 'M-ORG'], ['资', 24, 'M-ORG'], ['发', 25, 'M-ORG'], ['展', 26, 'M-ORG'], ['股', 27, 'M-ORG'], ['份', 28, 'M-ORG'], ['有', 29, 'M-ORG'], ['限', 30, 'M-ORG'], ['公', 31, 'M-ORG'], ['司', 32, 'E-ORG'], ['独', 33, 'B-TITLE'], ['立', 34, 'M-TITLE'], ['董', 35, 'M-TITLE'], ['事', 36, 'E-TITLE'], ['南', 38, 'B-ORG'], ['京', 39, 'M-ORG'], ['新', 40, 'M-ORG'], ['街', 41, 'M-ORG'], ['口', 42, 'M-ORG'], ['百', 43, 'M-ORG'], ['货', 44, 'M-ORG'], ['商', 45, 'M-ORG'], ['店', 46, 'M-ORG'], ['股', 47, 'M-ORG'], ['份', 48, 'M-ORG'], ['有', 49, 'M-ORG'], ['限', 50, 'M-ORG'], ['公', 51, 'M-ORG'], ['司', 52, 'E-ORG'], ['独', 53, 'B-TITLE'], ['立', 54, 'M-TITLE'], ['董', 55, 'M-TITLE'], ['事', 5

[['新', 10, 'B-ORG'], ['疆', 11, 'M-ORG'], ['天', 12, 'M-ORG'], ['山', 13, 'M-ORG'], ['毛', 14, 'M-ORG'], ['纺', 15, 'M-ORG'], ['织', 16, 'M-ORG'], ['股', 17, 'M-ORG'], ['份', 18, 'M-ORG'], ['有', 19, 'M-ORG'], ['限', 20, 'M-ORG'], ['公', 21, 'M-ORG'], ['司', 22, 'E-ORG'], ['职', 23, 'B-TITLE'], ['工', 24, 'M-TITLE'], ['监', 25, 'M-TITLE'], ['事', 26, 'E-TITLE']]
2007年3月至今任新疆天山毛纺织股份有限公司职工监事。
[]
[['陈', 0, 'B-NAME'], ['寿', 1, 'E-NAME'], ['中', 5, 'B-CONT'], ['国', 6, 'M-CONT'], ['国', 7, 'M-CONT'], ['籍', 8, 'E-CONT'], ['中', 10, 'B-ORG'], ['国', 11, 'M-ORG'], ['科', 12, 'M-ORG'], ['技', 13, 'M-ORG'], ['大', 14, 'M-ORG'], ['学', 15, 'E-ORG'], ['理', 16, 'B-PRO'], ['学', 17, 'E-PRO'], ['硕', 18, 'B-EDU'], ['士', 19, 'E-EDU'], ['上', 21, 'B-ORG'], ['海', 22, 'M-ORG'], ['交', 23, 'M-ORG'], ['通', 24, 'M-ORG'], ['大', 25, 'M-ORG'], ['学', 26, 'E-ORG'], ['E', 27, 'B-EDU'], ['M', 28, 'M-EDU'], ['B', 29, 'M-EDU'], ['A', 30, 'E-EDU'], ['高', 32, 'B-TITLE'], ['级', 33, 'M-TITLE'], ['工', 34, 'M-TITLE'], ['程', 35, 'M-TITLE'], ['师', 36, 

...

[]
[['江', 7, 'B-ORG'], ['西', 8, 'M-ORG'], ['赣', 9, 'M-ORG'], ['粤', 10, 'M-ORG'], ['高', 11, 'M-ORG'], ['速', 12, 'M-ORG'], ['公', 13, 'M-ORG'], ['路', 14, 'M-ORG'], ['股', 15, 'M-ORG'], ['份', 16, 'M-ORG'], ['有', 17, 'M-ORG'], ['限', 18, 'M-ORG'], ['公', 19, 'M-ORG'], ['司', 20, 'E-ORG'], ['独', 21, 'B-TITLE'], ['立', 22, 'M-TITLE'], ['董', 23, 'M-TITLE'], ['事', 24, 'E-TITLE'], ['公', 26, 'B-ORG'], ['司', 27, 'E-ORG'], ['独', 28, 'B-TITLE'], ['立', 29, 'M-TITLE'], ['董', 30, 'M-TITLE'], ['事', 31, 'E-TITLE']]
现已退休,现任江西赣粤高速公路股份有限公司独立董事、公司独立董事。
[]
[['张', 0, 'B-NAME'], ['崇', 1, 'M-NAME'], ['滨', 2, 'E-NAME'], ['中', 8, 'B-CONT'], ['国', 9, 'M-CONT'], ['国', 10, 'M-CONT'], ['籍', 11, 'E-CONT']]
张崇滨先生,男,中国国籍,
[]
[['大', 10, 'B-EDU'], ['学', 11, 'M-EDU'], ['学', 12, 'M-EDU'], ['历', 13, 'E-EDU'], ['陕', 18, 'B-ORG'], ['西', 19, 'M-ORG'], ['省', 20, 'M-ORG'], ['旅', 21, 'M-ORG'], ['游', 22, 'M-ORG'], ['局', 23, 'E-ORG'], ['W', 25, 'B-ORG'], ['e', 26, 'M-ORG'], ['i', 27, 'M-ORG'], ['j', 28, 'M-ORG'], ['i', 29, 'M-ORG'], ['a',

[['魏', 0, 'B-NAME'], ['永', 1, 'M-NAME'], ['新', 2, 'E-NAME'], ['硕', 6, 'B-EDU'], ['士', 7, 'M-EDU'], ['研', 8, 'M-EDU'], ['究', 9, 'M-EDU'], ['生', 10, 'E-EDU'], ['高', 12, 'B-TITLE'], ['级', 13, 'M-TITLE'], ['经', 14, 'M-TITLE'], ['济', 15, 'M-TITLE'], ['师', 16, 'E-TITLE']]
魏永新,男,硕士研究生,高级经济师。
[]
[['武', 2, 'B-ORG'], ['汉', 3, 'M-ORG'], ['车', 4, 'M-ORG'], ['站', 5, 'M-ORG'], ['路', 6, 'M-ORG'], ['商', 7, 'M-ORG'], ['场', 8, 'M-ORG'], ['房', 9, 'M-ORG'], ['地', 10, 'M-ORG'], ['产', 11, 'M-ORG'], ['开', 12, 'M-ORG'], ['发', 13, 'M-ORG'], ['公', 14, 'M-ORG'], ['司', 15, 'E-ORG'], ['总', 16, 'B-TITLE'], ['经', 17, 'M-TITLE'], ['理', 18, 'M-TITLE'], ['助', 19, 'M-TITLE'], ['理', 20, 'E-TITLE'], ['武', 22, 'B-ORG'], ['汉', 23, 'M-ORG'], ['国', 24, 'M-ORG'], ['有', 25, 'M-ORG'], ['资', 26, 'M-ORG'], ['产', 27, 'M-ORG'], ['经', 28, 'M-ORG'], ['营', 29, 'M-ORG'], ['公', 30, 'M-ORG'], ['司', 31, 'E-ORG'], ['资', 32, 'B-TITLE'], ['产', 33, 'M-TITLE'], ['部', 34, 'M-TITLE'], ['副', 35, 'M-TITLE'], ['经', 36, 'M-TITLE'], ['理', 37, 'E-TITLE

[['中', 12, 'B-ORG'], ['山', 13, 'M-ORG'], ['市', 14, 'M-ORG'], ['雅', 15, 'M-ORG'], ['居', 16, 'M-ORG'], ['乐', 17, 'M-ORG'], ['房', 18, 'M-ORG'], ['地', 19, 'M-ORG'], ['产', 20, 'M-ORG'], ['开', 21, 'M-ORG'], ['发', 22, 'M-ORG'], ['有', 23, 'M-ORG'], ['限', 24, 'M-ORG'], ['公', 25, 'M-ORG'], ['司', 26, 'E-ORG'], ['销', 28, 'B-TITLE'], ['售', 29, 'M-TITLE'], ['经', 30, 'M-TITLE'], ['理', 31, 'E-TITLE']]
1993年至1998年,中山市雅居乐房地产开发有限公司任销售经理;
[]
[['广', 12, 'B-ORG'], ['州', 13, 'M-ORG'], ['市', 14, 'M-ORG'], ['花', 15, 'M-ORG'], ['都', 16, 'M-ORG'], ['雅', 17, 'M-ORG'], ['居', 18, 'M-ORG'], ['乐', 19, 'M-ORG'], ['房', 20, 'M-ORG'], ['地', 21, 'M-ORG'], ['产', 22, 'M-ORG'], ['开', 23, 'M-ORG'], ['发', 24, 'M-ORG'], ['有', 25, 'M-ORG'], ['限', 26, 'M-ORG'], ['公', 27, 'M-ORG'], ['司', 28, 'E-ORG'], ['高', 30, 'B-TITLE'], ['级', 31, 'M-TITLE'], ['经', 32, 'M-TITLE'], ['理', 33, 'E-TITLE']]
2001年至2003年,广州市花都雅居乐房地产开发有限公司任高级经理;
[]
[['广', 12, 'B-ORG'], ['州', 13, 'M-ORG'], ['市', 14, 'M-ORG'], ['花', 15, 'M-ORG'], ['都', 16, 'M-ORG'], ['区',

[['墨', 0, 'B-ORG'], ['江', 1, 'M-ORG'], ['生', 2, 'M-ORG'], ['物', 3, 'M-ORG'], ['镍', 4, 'M-ORG'], ['业', 5, 'M-ORG'], ['有', 6, 'M-ORG'], ['限', 7, 'M-ORG'], ['公', 8, 'M-ORG'], ['司', 9, 'E-ORG'], ['董', 10, 'B-TITLE'], ['事', 11, 'M-TITLE'], ['长', 12, 'E-TITLE'], ['总', 14, 'B-TITLE'], ['经', 15, 'M-TITLE'], ['理', 16, 'E-TITLE']]
墨江生物镍业有限公司董事长、总经理。
[]
[['云', 2, 'B-ORG'], ['锡', 3, 'M-ORG'], ['元', 4, 'M-ORG'], ['江', 5, 'M-ORG'], ['镍', 6, 'M-ORG'], ['业', 7, 'M-ORG'], ['有', 8, 'M-ORG'], ['限', 9, 'M-ORG'], ['责', 10, 'M-ORG'], ['任', 11, 'M-ORG'], ['公', 12, 'M-ORG'], ['司', 13, 'E-ORG'], ['总', 14, 'B-TITLE'], ['经', 15, 'M-TITLE'], ['理', 16, 'E-TITLE']]
现任云锡元江镍业有限责任公司总经理。
[]
[['吴', 0, 'B-NAME'], ['光', 1, 'E-NAME'], ['汉', 5, 'B-RACE'], ['族', 6, 'E-RACE']]
吴光,男,汉族,
[]
[['中', 10, 'B-TITLE'], ['共', 11, 'M-TITLE'], ['党', 12, 'M-TITLE'], ['员', 13, 'E-TITLE'], ['教', 15, 'B-TITLE'], ['授', 16, 'E-TITLE'], ['博', 18, 'B-TITLE'], ['士', 19, 'M-TITLE'], ['生', 20, 'M-TITLE'], ['导', 21, 'M-TITLE'], ['师', 22, 'E-TITLE']

[['深', 1, 'B-ORG'], ['圳', 2, 'M-ORG'], ['市', 3, 'M-ORG'], ['卡', 4, 'M-ORG'], ['金', 5, 'M-ORG'], ['亚', 6, 'M-ORG'], ['珠', 7, 'M-ORG'], ['宝', 8, 'M-ORG'], ['有', 9, 'M-ORG'], ['限', 10, 'M-ORG'], ['公', 11, 'M-ORG'], ['司', 12, 'E-ORG'], ['董', 13, 'B-TITLE'], ['事', 14, 'M-TITLE'], ['长', 15, 'E-TITLE']]
任深圳市卡金亚珠宝有限公司董事长;
[]
[['深', 0, 'B-ORG'], ['圳', 1, 'M-ORG'], ['仲', 2, 'M-ORG'], ['裁', 3, 'M-ORG'], ['委', 4, 'M-ORG'], ['员', 5, 'M-ORG'], ['会', 6, 'E-ORG'], ['仲', 7, 'B-TITLE'], ['裁', 8, 'M-TITLE'], ['员', 9, 'E-TITLE']]
深圳仲裁委员会仲裁员;
[]
[['深', 0, 'B-ORG'], ['圳', 1, 'M-ORG'], ['市', 2, 'M-ORG'], ['深', 3, 'M-ORG'], ['宝', 4, 'M-ORG'], ['实', 5, 'M-ORG'], ['业', 6, 'M-ORG'], ['股', 7, 'M-ORG'], ['份', 8, 'M-ORG'], ['有', 9, 'M-ORG'], ['限', 10, 'M-ORG'], ['公', 11, 'M-ORG'], ['司', 12, 'E-ORG'], ['独', 13, 'B-TITLE'], ['立', 14, 'M-TITLE'], ['董', 15, 'M-TITLE'], ['事', 16, 'E-TITLE']]
深圳市深宝实业股份有限公司独立董事;
[]
[['杭', 16, 'B-ORG'], ['州', 17, 'M-ORG'], ['天', 18, 'M-ORG'], ['目', 19, 'M-ORG'], ['山', 20, 'M-ORG'], ['药', 2

## NewsCN

In [5]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/NewsCN/'

Corpus2GroupMethod = '.char'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = True

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': ''
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/NewsCN/demo.dev.char
上海浦东开发与法制建设同步
[['上海', 0, 2, 'GPE'], ['浦东', 2, 4, 'GPE']]
新华社上海二月十日电(记者谢金虎、张持坚)
[['新华社', 0, 3, 'ORG'], ['上海', 3, 5, 'GPE'], ['谢金虎', 13, 16, 'PER'], ['张持坚', 17, 20, 'PER']]
上海浦东近年来颁布实行了涉及经济、贸易、建设、规划、科技、文教等领域的七十一件法规性文件,确保了浦东开发的有序进行。
[['上海', 0, 2, 'GPE'], ['浦东', 2, 4, 'GPE'], ['浦东', 48, 50, 'GPE']]
浦东开发开放是一项振兴上海,建设现代化经济、贸易、金融中心的跨世纪工程,因此大量出现的是以前不曾遇到过的新情况、新问题。
[['浦东', 0, 2, 'GPE'], ['上海', 11, 13, 'GPE']]
对此,浦东不是简单的采取“干一段时间,等积累了经验以后再制定法规条例”的做法,而是借鉴发达国家和深圳等特区的经验教训,聘请国内外有关专家学者,积极、及时地制定和推出法规性文件,使这些经济活动一出现就被纳入法制轨道。
[['浦东', 3, 5, 'GPE'], ['深圳', 48, 50, 'GPE'], ['特区', 51, 53, 'LOC']]
去年初浦东新区诞生的中国第一家医疗机构药品采购服务中心,正因为一开始就比较规范,运转至今,成交药品一亿多元,没有发现一例回扣。
[['浦东', 3, 5, 'GPE'], ['中国', 10, 12, 'GPE']]
建筑是开发浦东的一项主要经济活动,这些年有数百家建筑公司、四千余个建筑工地遍布在这片热土上。
[['浦东', 5, 7, 'GPE']]
为规范建筑行为,防止出现无序现象,新区管委会根据国家和上海市的有关规定,结合浦东开发实际,及时出台了一系列规范建设市场的文件,其中包括工程施工招投标管理办法、拆迁工作若干规定、整治违章建筑实施办法、通信设施及管线配套建设意见、建设工地施工环境管理暂行办法等,基本做到了每个环节都有明确而又具体的规定。
[['上海市', 27, 30, 'GPE'], ['浦东', 38, 40, 'GPE']]
建筑公司进区,有关部

直到今天,玉市、夜市里,三脚蟾蜍仍到处可见,只是,随着哈日风席卷台湾,招财猫已然当道,蟾蜍造型物在市场上的盛况不再。
[['哈日风', 27, 30, 'GPE'], ['台湾', 32, 34, 'GPE']]
倒是大肚能容的弥勒佛由于造型讨喜,还能与招财猫平起平坐,被左一个、右一个的摆在店里,市场上,各式陶瓷、木雕、玉石材质的弥勒佛,卖相也不差。
[['弥勒佛', 7, 10, 'PER'], ['弥勒佛', 59, 62, 'PER']]
传说弥勒佛曾化身布袋和尚,四处化缘来的东西,就往背后布袋里装,而他取自十方,散自十方,这只手进,呵呵呵笑的另一只手就布施给了人家。
[['弥勒佛', 2, 5, 'PER']]
仔细看看人家店里、家里的弥勒佛,背上常常背着圆鼓鼓的大布袋,至于这金山银山般的布袋里装的是什么?可就随现代人发挥想像了。
[['弥勒佛', 12, 15, 'PER']]
摇钱树、金鸡母滚钱财
[]
至于因为与钱财谐音就成了金鸡母的,除了金钱豹饰物,前几年,台湾人还曾一窝蜂流行买「棺材」,大商家柜台上就摆个尺长的朱漆棺、小市民则随身携带配有迷你棺材的钥匙圈,这么没忌讳的与棺材为伍,不都贪它个——升官发财!
[]
针对各行各业问起来,个别招财物的买气还真是人家逢低它走高,往往成了店里的摇钱树。
[]
不就因为景气看坏,开花店、专卖切花的老板王年丰近来唉声连连,平日常做影视与媒体业者生意的他,直言娱乐界的公关费少掉一半,他的花儿生意也黯淡许多。
[['王年丰', 20, 23, 'PER']]
可同样是花店,离他几步路远的同业,店员却直说生意不坏,放眼一看,小小的店门口前,簇拥着一盆盆珠光宝气的元宝树,元宝造型的花盆里,长着元宝造型的果实,枝条上还绑着穿有珠子的红色蝴蝶结。
[]
另一头则并列着枝条匀长的金钱树,圆而厚实的叶子从底部往上对生,如一串串铜钱。
[]
这两种进口植物因长相而贵,几乎没有店家喊得出它们的「本名」,反正买的人也都不问如何称呼,只要说能招财,就这么一盆盆立时被搬回家。
[]
「这是银行的最爱,」台北市八德路上的一品花店店员果断的表示。
[]
景气不佳,生意难做,街头店面不时重新装潢、换头家,这一盆盆价格不高又吉祥的招财树,就成了朋友新店开张的最佳礼品!
[]
说的夸张点,就靠黄金葛、马拉巴栗、元宝树、开运竹等几种招财植物,招子亮的花

并由于祖父在京城为官,每年都会买些名贵考究的文墨物品回来,王壮为不仅从小就对各地名家的笔墨纸砚了若指掌,当其他孩子顾着游戏时,他则兴致勃勃地躲在家族的藏书室里找书看,而且一有机会进去,就不出来了。
[['京城', 6, 8, 'GPE'], ['王壮为', 29, 32, 'PER']]
和一般中国孩子一样,王壮为亦由学颜真卿字体开始,因为颜体字有骨有肉,端正严谨、中庸大方。
[['王壮为', 10, 13, 'PER'], ['颜真卿', 16, 19, 'PER'], ['颜体字', 26, 29, 'PER']]
在临摹诸多名家书帖之后,青春少年时候,则以王羲之的行书及草书,最契合己意,而以之为书风的根本主体。
[['王羲之', 21, 24, 'PER']]
烽火儿女情
[]
二十岁时,王壮为离家进入北平的私立京华美专就读,在那一个东西文化刚开始碰触的年代,一如其他的青年人,他汲汲于吸收西洋文化,除了油画、雕刻、蜡笔画,包括电影、新文艺等西方艺术无不爱好。
[['王壮为', 5, 8, 'PER'], ['北平', 12, 14, 'GPE'], ['私立京华美专', 15, 21, 'ORG']]
也或许是童年时光,见识过中国艺术的精髓,日后的王壮为并未朝西洋艺术发展,反而以一个道地的中国文人生活为其终身选择。
[['王壮为', 23, 26, 'PER']]
民国二十六年,芦沟桥事变,北平为日本人所占领,才订了亲的王壮为,秉着燕赵男儿的英勇豪气决定投笔从戎,尔后加入罗卓英将军的十九军团,跟着远征军到蛮荒瘴疠的缅甸行军三个月,西走印度才得以安全归来。
[['北平', 13, 15, 'GPE'], ['王壮为', 28, 31, 'PER'], ['罗卓英', 54, 57, 'PER'], ['十九军团', 60, 64, 'ORG'], ['缅甸', 76, 78, 'GPE'], ['印度', 86, 88, 'GPE']]
当时,与他才成亲不久的张润生将军之女张明随,因为打听不到新婚夫婿的消息,竟然一个人带着另一对也要寻夫的母子,就由江西往西南方向前去,同样有着北方儿女的英气。
[['张润生', 11, 14, 'PER'], ['张明随', 18, 21, 'PER'], ['江西', 56, 58, 'GPE']]
「多年后,我翻阅他当时的日记

隐地在出版界的地位之重不言而喻,创立只出文学类书的尔雅,二十五年来出版了许多影响深远的好书,从《开放的人生》(王鼎钧)、《三更有梦书当枕》(琦君)到《代马输卒手记》(张拓芜)、《文化苦旅》(余秋雨)等等。
[['隐地', 0, 2, 'PER'], ['尔雅', 25, 27, 'ORG'], ['王鼎钧', 55, 58, 'PER'], ['琦君', 70, 72, 'PER'], ['张拓芜', 83, 86, 'PER'], ['余秋雨', 95, 98, 'PER']]
一向都帮别人出书的隐地,怎知这回写出自个儿的故事,竟引起读者、文友广大回响。
[['隐地', 9, 11, 'PER']]
隐地爽朗地表示:「我把别人逗哭了,自己却笑了」。
[['隐地', 0, 2, 'PER']]
若说书写治疗可以挥去幼时自卑、孤独、落寞的经验,成书后心情倒也轻松快活,无怪乎隐地说他现在是不愉快环境中最快乐的人。
[['隐地', 39, 41, 'PER']]
走在人声鼎沸、车如流水的街上,克难街、上海路、三军球场、明星咖啡屋,早已随着葛兰「嗨曼波,曼波啦……」歌声飘然远去。
[['葛兰', 38, 40, 'PER']]
昔时熟识的场景也随风散去,让身为老台北人的隐地有些惘然,自喻是工业社会中的农业人,守着文学,坚持精致出版,为小众出好书的隐地,笑谈未来,计画写一部长篇小说,就让喜爱隐地的读者,共同期待着这一天的到来。
[['隐地', 21, 23, 'PER'], ['隐地', 60, 62, 'PER'], ['隐地', 82, 84, 'PER']]
文末,谨以诗作一首,送给资深出版人——隐地。
[['隐地', 19, 21, 'PER']]
隐身于市,心地朴真。
[]
壮年诗情,爱书痴人。
[]
p.104
[]
书名:涨潮日
[]
作者:隐地
[['隐地', 3, 5, 'PER']]
出版:尔雅出版社
[['尔雅出版社', 3, 8, 'ORG']]
出版日期:2000年11月
[]
地址:台北市厦门街113巷33之1号
[]
电话:(02)23654036
[]
定价:200元
[]
p.105
[]
不管路人行脚匆匆,隐地依然坚持驻足在文学花园里耕耘。
[['隐地', 9, 11, 'PER']]
(邱瑞金摄)
[['邱瑞金', 1, 4, 'PER'

文部位于西藏高原西北部,平均海拔近5000米,曾被视为“生命的禁区”,所谓的“无人区”就在这一带的千里荒原上。
[['文部', 0, 2, 'GPE'], ['西藏', 4, 6, 'GPE'], ['西北部', 8, 11, 'LOC']]
经过13年的开发建设,文部12万平方公里的草原,旧貌变新颜。
[['文部', 11, 13, 'GPE']]
从无人到有人,从游牧到定居,如今这里已成了牛羊满山坡的新牧区。
[]
办事处所在地已经成为颇具规模的小城镇,条件相当不错。
[]
访问团的成员们决定到牧民家里看望。
[]
当年曾参加过开发建设“无人区”、现任那曲地区行署专员的土登才旺和地委副书记明加带领各县领导,驱车行程800多公里,访问了7个乡的牧民家庭。
[['那曲', 18, 20, 'GPE'], ['土登才旺', 27, 31, 'PER'], ['地委', 32, 34, 'ORG'], ['明加', 37, 39, 'PER'], ['乡', 62, 63, 'GPE']]
当他们进入申亚乡一村牧民索次家时,45岁的主人倒酥油茶招待大家。
[['申亚乡', 5, 8, 'LOC'], ['索次', 12, 14, 'PER']]
索次家的摆设令来访者惊讶不已:5间土木结构的房舍,既有双层玻璃的客厅,又有装满羊肉、酥油的仓库,还有一间摆有烟、酒、糖等日用品。
[['索次', 0, 2, 'PER']]
客厅里的钢丝床上铺有龙凤呈祥的卡垫,刻有传统图案的一对藏柜上放着两部收录机。
[]
文部办事处主任加央多吉说,索次一家在文部牧民中属中等生活水平。
[['文部', 0, 2, 'GPE'], ['加央多吉', 7, 11, 'PER'], ['索次', 13, 15, 'PER'], ['文部', 18, 20, 'GPE']]
阿索乡牧民阿珍一家15口人,有1800只羊,50头牦牛,已向国家出售羊毛750公斤、山羊绒50公斤、酥油25公斤,现有存款达1·5万多元。
[['阿索乡', 0, 3, 'LOC'], ['阿珍', 5, 7, 'PER']]
访问团来到中仓乡巴拉一家。
[['中仓乡', 5, 8, 'LOC'], ['巴拉', 8, 10, 'PER']]
巴拉家已有300多只羊,而在这里却只能算是“特困户”。
[['巴拉', 0, 2, 'PE

...

卸任后,尼克松先生继续关心中美关系,为维护和发展两国关系作出了重要贡献。
[['尼克松', 4, 7, 'PER'], ['中', 13, 14, 'GPE'], ['美', 14, 15, 'GPE']]
我们深信,在中美双方的共同努力下,尼克松先生生前所关心的中美关系一定会得到进一步改善和发展。
[['中', 6, 7, 'GPE'], ['美', 7, 8, 'GPE'], ['尼克松', 17, 20, 'PER'], ['中', 28, 29, 'GPE'], ['美', 29, 30, 'GPE']]
谨向尼克松先生的家属致以亲切的慰问。
[['尼克松', 2, 5, 'PER']]
天津对台合作前景广阔
[['天津', 0, 2, 'GPE'], ['台', 3, 4, 'GPE']]
新华社天津九月一日电(记者刘庆禄、窦合义)
[['新华社', 0, 3, 'ORG'], ['天津', 3, 5, 'GPE'], ['刘庆禄', 13, 16, 'PER'], ['窦合义', 17, 20, 'PER']]
天津市市长张立昌在最近召开的天津市对台经济工作会议上说,天津与台湾产业结构相似,技术相近,相互间有很强的互补性,双方合作前景广阔,大有可为。
[['天津市', 0, 3, 'GPE'], ['张立昌', 5, 8, 'PER'], ['天津市', 14, 17, 'GPE'], ['台', 18, 19, 'GPE'], ['天津', 28, 30, 'GPE'], ['台湾', 31, 33, 'GPE']]
据统计,到今年七月底,天津市共批准台商投资企业八百七十一家,协议投资额九亿多美元,其中台资占百分之七十三点八。
[['天津市', 11, 14, 'GPE']]
台资企业数和协议投资额分别列全市外商投资企业的第三位和第四位。
[]
一批投资额较大、技术较高的台资企业在天津落户。
[['天津', 18, 20, 'GPE']]
在已开业的台资企业中,九成以上盈利。
[]
张立昌说,今后天津市将继续努力为台商创造一个有利可图,有钱可赚的投资环境。
[['张立昌', 0, 3, 'PER'], ['天津市', 7, 10, 'GPE']]
根据“同等优先、适当放宽”的原则,天津市将调整有关政策,以增强对台商的吸引力。
[['天津市', 17, 

首先要保障群众的生活供水,采取综合措施,稳定群众情绪,保持社会安定。
[]
要采取果断措施,防止淮河流域再次发生重大水污染事故。
[['淮河', 10, 12, 'LOC']]
宋健重申,在淮河流域范围内,禁止新建小造纸、小化工、小制革等污染严重的项目。
[['宋健', 0, 2, 'PER'], ['淮河', 6, 8, 'LOC']]
对所有向淮河流域河流排污的企业,要进行限期治理,最迟于一九九七年底停止向河流排污。
[['淮河', 4, 6, 'LOC']]
宋健要求完善淮河流域的法制建设,加强环保执法力度。
[['宋健', 0, 2, 'PER'], ['淮河', 6, 8, 'LOC']]
对违反有关环保法规造成严重环境污染的责任者要严加惩处,对特别恶劣的犯罪行为要依法严厉打击。
[]
钱其琛会见朝鲜政府特使
[['钱其琛', 0, 3, 'PER'], ['朝鲜', 5, 7, 'GPE'], ['政府', 7, 9, 'ORG']]
新华社北京九月一日电
[['新华社', 0, 3, 'ORG'], ['北京', 3, 5, 'GPE']]
国务院副总理兼外交部长钱其琛今天在钓鱼台国宾馆会见朝鲜政府特使、外交部副部长宋浩京。
[['国务院', 0, 3, 'ORG'], ['钱其琛', 11, 14, 'PER'], ['钓鱼台', 17, 20, 'LOC'], ['朝鲜', 25, 27, 'GPE'], ['政府', 27, 29, 'ORG'], ['外交部', 32, 35, 'ORG'], ['宋浩京', 38, 41, 'PER']]
宋浩京转达了朝鲜领导人对中国领导人的亲切问候,代表朝方对中国党政领导人和人民哀悼金日成主席逝世表示深切谢意。
[['宋浩京', 0, 3, 'PER'], ['朝鲜', 6, 8, 'GPE'], ['中国', 12, 14, 'GPE'], ['中国', 28, 30, 'GPE'], ['金日成', 40, 43, 'PER']]
钱其琛感谢宋浩京转达朝鲜领导人的问候,表示相信朝鲜人民将紧密团结在以金正日为首的朝鲜劳动党中央周围,继续完成金日成主席的遗愿。
[['钱其琛', 0, 3, 'PER'], ['宋浩京', 5, 8, 'PER'], ['朝鲜', 10, 12, 'GPE'], ['朝鲜',

此后,各地纷纷成立残疾人体育组织。
[]
中国智残人、聋人体育协会也于1985年成立。
[['中国', 0, 2, 'GPE']]
有了组织、领导,比赛、评比等活动日益频繁,残疾人参加体育活动的积极性空前高涨,运动水平不断提高,后备力量也在增强。
[]
合肥、唐山、广州三届全国残疾人运动会,规模越来越大,水平越来越高,引起社会越来越广泛的关注和支持,并吸引了越来越多的残疾人参加体育锻炼。
[['合肥', 0, 2, 'GPE'], ['唐山', 3, 5, 'GPE'], ['广州', 6, 8, 'GPE']]
自1984年起,中国残疾人体育协会相继成为国际残疾人奥委会等8个国际残疾人体育组织的成员,与一些国家和地区的残疾人体育组织建立了联系,并登上残疾人奥运会等国际赛场。
[['中国残疾人体育协会', 8, 17, 'ORG'], ['国际残疾人奥委会', 21, 29, 'ORG']]
在比赛与交往中,中国残疾人显示了非凡的体育才能,取得优异的成绩,增进了与各个国家和地区残疾人的友谊,为世界残疾人事业的发展作出了贡献。
[['中国', 8, 10, 'GPE']]
于1991年颁布实施的《中华人民共和国残疾人保障法》规定,要组织和扶持残疾人开展群众性的体育活动,举办特殊体育运动会,参加重大国际性比赛和交流。
[]
法律的保障已经并将继续促进中国残疾人体育事业更加蓬勃地发展。
[['中国', 13, 15, 'GPE']]
第六届远南运动会在北京举行,必将促使整个社会进一步了解并支持残疾人事业,大大增强中国人的助残意识,使更多的残疾人投入体育活动,从而对中国残疾人体育事业的发展产生巨大的影响。
[['北京', 9, 11, 'GPE'], ['中国', 66, 68, 'GPE']]
诚然,中国残疾人体育事业的基础并不雄厚,普及面还不很广,与一些发达国家相比还有不小的差距。
[['中国', 3, 5, 'GPE']]
但是,远南运动会掀起的“冲击波”,预示着这项文明、进步的事业前景是美好的。
[]
美国网球公开赛进入第三天
[['美国', 0, 2, 'GPE']]
新华社北京9月1日电
[['新华社', 0, 3, 'ORG'], ['北京', 3, 5, 'GPE']]
据外电报道,美国网球公开赛8月31日进入第三天,被左脚踝关节伤痛困扰了一个多月的上届冠军

「这也有历史的原因,」上届澳门留台大专校友会理事长、也是澳门特别行政区第一届推选委员、筹备委员的区金蓉表示,在一九八○年代以前,澳门并未有大专院校的设立,中学生若要继续升学,必须负笈海外。
[['澳门', 13, 15, 'GPE'], ['留台大专校友会', 15, 22, 'ORG'], ['澳门', 28, 30, 'GPE'], ['区金蓉', 48, 51, 'PER'], ['澳门', 64, 66, 'GPE']]
而在一九八○年代中期以前,大陆大专院校并未开放对外招收学生,香港大学少又难进,积极向海外招生的台湾各大学自然成为许多澳门同学的选择。
[['大陆', 13, 15, 'GPE'], ['香港', 30, 32, 'GPE'], ['台湾', 47, 49, 'GPE'], ['澳门', 58, 60, 'GPE']]
「从投资报酬率来说,到台湾念书很上算,」澳门市政厅清洁暨环保处代处长、台南成大土木工程系毕业的汪云说,台湾的大学师资好、设备也不错,澳门的同学在台湾可以学到不少东西,对将来的工作很有帮助。
[['台湾', 11, 13, 'GPE'], ['澳门市政厅清洁暨环保处', 20, 31, 'ORG'], ['台南成大土木工程系', 35, 44, 'ORG'], ['汪云', 47, 49, 'PER'], ['台湾', 51, 53, 'GPE'], ['澳门', 66, 68, 'GPE'], ['台湾', 72, 74, 'GPE']]
与香港殖民政府不同,澳葡政府承认台湾各大学的学历,八○年代中期,香港澳门转移时间抵定,澳门才开始为特区政府「澳人治澳」铺路。
[['香港', 1, 3, 'GPE'], ['澳葡', 10, 12, 'GPE'], ['台湾', 16, 18, 'GPE'], ['香港澳门', 32, 36, 'GPE'], ['澳门', 43, 45, 'GPE']]
澳门政府「公务员本地化」的政策一开,许多在社会上已经奋斗多年的留台校友,从民间转入政府,成为政府中坚份子。
[['澳门', 0, 2, 'GPE'], ['台', 32, 33, 'GPE']]
「海中若无鱼,小虾变大鱼,」一九七三年于台大企管系毕业的梁金泉开玩笑地说,比起八○年代中期以后才到大陆念书的许多澳门同学,留台校友因为回来得早,进

聚纺的微多孔透湿防水加工,是靠着涂布配方中的特殊水溶性粒子,在布面上形成微孔,汗气可以从微孔中排出而不会被布料吸收,有效改善了以往亲水型防水衣在雨中穿久了就黏在身上的不舒服感觉;而聚纺更进一步和纺研合作,研发兼具微多孔及亲水两种特性的防水布种。
[['聚纺', 0, 2, 'ORG'], ['聚纺', 90, 92, 'ORG']]
目前来下单的多半是欧美客户,经过加工制成登山装、高尔夫球装等高级休闲衣,一件单价可以达到八千元台币。
[]
和大清桦一样,聚纺也希望自己的产品能带动国内产业升级。
[['大清桦', 1, 4, 'ORG'], ['聚纺', 7, 9, 'ORG']]
「二十一世纪是休闲时代,台湾的运动服、休闲服布种产量一向高居全球第一,如果能具备更高级的透湿、透气及防水功能,可以大大提高附加价值,」蔡秋雄说。
[['台湾', 12, 14, 'GPE']]
目前聚纺加工后的透湿防水布比日本同级进口货便宜三分之一,希望能吸引国内业者多多利用,发挥台湾产业惯有的蚂蚁雄兵、众志成城的优势。
[['日本', 14, 16, 'GPE']]
绑在业绩上的创意
[]
和透湿防水有异曲同工之妙的新种热门纤维,是以「异形断面纤维纺纱技术」为基础的吸湿排汗纤维。
[]
国内领先量产吸湿排汗纤维的中兴纺织总经理商武解释,以往的纤维是一根根圆滚滚的细丝,断面切出来是一个个圆形。
[['中兴纺织', 13, 17, 'ORG'], ['商武', 20, 22, 'PER']]
然而将纤维的物理表面改变后,纤维断面可以呈现如十字形、Y字型等各种不规则形状,形成纤维间的毛细孔。
[]
不仅人体的汗气可以瞬间排出体外,还有调节体温、洗后快干等好处。
[]
只是这种纤维的特性在纺纱、织布、染整等各个阶段都很容易被破坏,因此要做出成品,需要的技术非常繁复。
[]
目前中兴已推出名为「COOLPLUS」的产品,和美商杜邦的同级产品「COOLMAX」打起对台。
[['中兴', 2, 4, 'ORG'], ['杜邦', 26, 28, 'ORG']]
「新纤维的开发,是纺织产业升级中最重要的一环,」商武表示。
[['商武', 24, 26, 'PER']]
然而他坦承,尽管国内新产品林林总总,追溯起来,其实都是国外某种产品的翻版,离真正的创新还有很长一段距离。
[]
只是台湾业者以往可能要费

## CoNLL-2003

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/CoNLL-2003/'

Corpus2GroupMethod = '.openNLP'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'iter'
TOKENLevel = 'word'

min_token_freq = 1

use_hyper = True

anno = 'conll_block'
anno_keywords = {
    'anno_sep': ' ',
    'connector': ' ', 
    'suffix': False,
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/CoNLL-2003/eng.train.openNLP
[]
[0]
-DOCSTART-
[]
[['EU', 0, 'I-ORG'], ['German', 2, 'I-MISC'], ['British', 6, 'I-MISC']]
[3]
EU rejects German call to boycott British lamb .
[]
[['Peter', 0, 'I-PER'], ['Blackburn', 1, 'I-PER']]
[2]
Peter Blackburn
[]
[['BRUSSELS', 0, 'I-LOC']]
[1]
BRUSSELS 1996-08-22
[]
[['European', 1, 'I-ORG'], ['Commission', 2, 'I-ORG'], ['German', 9, 'I-MISC'], ['British', 15, 'I-MISC']]
[4]
The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
[]
[['Germany', 0, 'I-LOC'], ['European', 5, 'I-ORG'], ['Union', 6, 'I-ORG'], ['Werner', 10, 'I-PER'], ['Zwingmann', 11, 'I-PER'], ['Britain', 23, 'I-LOC']]
[6]
Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
[]
[['

TypeError: exceptions must derive from BaseException

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/NLPBA2004/'

Corpus2GroupMethod = '.iob2'

Group2TextMethod   = 'block'

Text2SentMethod  = 'whole'

Sent2TokenMethod = ' '
TOKENLevel = 'word'

min_token_freq = 1

use_hyper = False

anno = 'conll_block'
anno_keywords = {
    'anno_sep': '\t',
    'connector': ' ', 
    'suffix': False,
}
BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


'File'
corpus/NLPBA2004/sampletest1.iob2
[['proinflammatory', 6, 'B-protein'], ['cytokine', 7, 'I-protein'], ['tumor', 9, 'B-protein'], ['necrosis', 10, 'I-protein'], ['factor-alpha', 11, 'I-protein'], ['interleukin-6', 13, 'B-protein'], ['interferon-gamma', 16, 'B-protein'], ['peripheral', 21, 'B-cell_type'], ['blood', 22, 'I-cell_type'], ['mononuclear', 23, 'I-cell_type'], ['cells', 24, 'I-cell_type']]
[0, 2, 5, 6, 7, 11]
High-dose growth hormone does not affect proinflammatory cytokine ( tumor necrosis factor-alpha , interleukin-6 , and interferon-gamma ) release from activated peripheral blood mononuclear cells or after minimal to moderate surgical stress .
[['proinflammatory cytokine', 6, 8, 'protein'], ['tumor necrosis factor-alpha', 9, 12, 'protein'], ['interleukin-6', 13, 14, 'protein'], ['interferon-gamma', 16, 17, 'protein'], ['peripheral blood mononuclear cells', 21, 25, 'cell_type']]

CITAnnoText

[['proinflammatory', 6, 'protein-B'], ['cytokine', 7, 'protein-E'], ['tumor',

IndexError: list index out of range

# Functions to Handle Labels

## strText and SSET

For each text, we must get a strText and SSET, which meet the following restriction.

Besides, we also need a strSents.

In [39]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''

print(strText)

strAnnoText = '''标注文本名称:/Users/zhangling/Documents/新标的数据530/529李选-已检查/Entity/patient4378.txt\n标注文本字数统计:87\n多发息肉\t3\t6\t疾病\n慢性\t16\t17\t修饰\n多发息肉\t30\t33\t疾病\n3月余\t34\t36\t修饰\n无阳性体征\t44\t48\t不确定\n'''

# BIOES

sep = '\t'
SSETText = [sset.split('\t') for sset in strAnnoText.split('\n') if sep in sset]


notZeroIndex = 1 
for idx, sset in enumerate(SSETText):
    data = (sset[0], int(sset[1]) - notZeroIndex, int(sset[2]), sset[-1])
    SSETText[idx] = data


# check
for sset in SSETText:
    try:
        assert strText[sset[1]: sset[2]] == sset[0]
    except:
        print('strText:', strText[sset[1] : sset[2]])
        print('SSETText:', sset[0])
        
SSETText

结肠多发息肉。
患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。


[('多发息肉', 2, 6, '疾病'),
 ('慢性', 15, 17, '修饰'),
 ('多发息肉', 29, 33, '疾病'),
 ('3月余', 33, 36, '修饰'),
 ('无阳性体征', 43, 48, '不确定')]

## getCITText

In [40]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''


SSETText = [ ('多发息肉', 2, 6, '疾病'),
             ('慢性', 15, 17, '修饰'),
             ('多发息肉', 29, 33, '疾病'),
             ('3月余', 33, 36, '修饰'),
             ('无阳性体征', 43, 48, '不确定')]

def getCITText(strText, SSETText, TOKENLevel='char'):
    len(SSETText) > 0 
    if TOKENLevel == 'char':
        for sset in SSETText:
            try:
                assert strText[sset[1]: sset[2]] == sset[0]
            except:
                print('strText:', strText[sset[1] : sset[2]])
                print('SSETText:', sset[0])
        CITAnnoText = []
        for sset in SSETText:
            # BIOES
            strAnno, s, e, tag = sset
            CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]
            CIT[-1][2] = tag + '-E'
            CIT[ 0][2] = tag + '-B'
            if len(CIT) == 1:
                CIT[0][2] = tag + '-S' 
            CITAnnoText.extend(CIT)

        # print(strAnnoText)
        CITText = [[char, idx, 'O'] for idx, char in enumerate(strText)]
        for citAnno in CITAnnoText:
            c, idx, t = citAnno
            assert CITText[idx][0] == c
            CITText[idx] = citAnno

    elif TOKENLevel == 'word':
        CITText = []
        for idx, sset in enumerate(SSETText):
            try:
                assert sset[0] == strText[idx]
            except:
                print(strText)[idx]
                print(sset[0])

            CITText.append(sset)
    return CITText


CITText = getCITText(strText, SSETText, TOKENLevel='char')
CITText

[['结', 0, 'O'],
 ['肠', 1, 'O'],
 ['多', 2, '疾病-B'],
 ['发', 3, '疾病-I'],
 ['息', 4, '疾病-I'],
 ['肉', 5, '疾病-E'],
 ['。', 6, 'O'],
 ['\n', 7, 'O'],
 ['患', 8, 'O'],
 ['中', 9, 'O'],
 ['老', 10, 'O'],
 ['年', 11, 'O'],
 ['男', 12, 'O'],
 ['性', 13, 'O'],
 [',', 14, 'O'],
 ['慢', 15, '修饰-B'],
 ['性', 16, '修饰-E'],
 ['病', 17, 'O'],
 ['程', 18, 'O'],
 ['。', 19, 'O'],
 [' ', 20, 'O'],
 ['因', 21, 'O'],
 ['“', 22, 'O'],
 ['体', 23, 'O'],
 ['检', 24, 'O'],
 ['发', 25, 'O'],
 ['现', 26, 'O'],
 ['大', 27, 'O'],
 ['肠', 28, 'O'],
 ['多', 29, '疾病-B'],
 ['发', 30, '疾病-I'],
 ['息', 31, '疾病-I'],
 ['肉', 32, '疾病-E'],
 ['3', 33, '修饰-B'],
 ['月', 34, '修饰-I'],
 ['余', 35, '修饰-E'],
 ['”', 36, 'O'],
 ['入', 37, 'O'],
 ['院', 38, 'O'],
 ['。', 39, 'O'],
 ['查', 40, 'O'],
 ['体', 41, 'O'],
 [':', 42, 'O'],
 ['无', 43, '不确定-B'],
 ['阳', 44, '不确定-I'],
 ['性', 45, '不确定-I'],
 ['体', 46, '不确定-I'],
 ['征', 47, '不确定-E'],
 ['。', 48, 'O']]

## getCITSents

In [2]:
strSents = ['结肠多发息肉。', '患中老年男性,慢性病程。', '因“体检发现大肠多发息肉3月余”入院。', '查体:无阳性体征。']

CITText  =  [['结', 0, 'O'],
             ['肠', 1, 'O'],
             ['多', 2, '疾病-B'],
             ['发', 3, '疾病-I'],
             ['息', 4, '疾病-I'],
             ['肉', 5, '疾病-E'],
             ['。', 6, 'O'],
             ['\n', 7, 'O'],
             ['患', 8, 'O'],
             ['中', 9, 'O'],
             ['老', 10, 'O'],
             ['年', 11, 'O'],
             ['男', 12, 'O'],
             ['性', 13, 'O'],
             [',', 14, 'O'],
             ['慢', 15, '修饰-B'],
             ['性', 16, '修饰-E'],
             ['病', 17, 'O'],
             ['程', 18, 'O'],
             ['。', 19, 'O'],
             [' ', 20, 'O'],
             ['因', 21, 'O'],
             ['“', 22, 'O'],
             ['体', 23, 'O'],
             ['检', 24, 'O'],
             ['发', 25, 'O'],
             ['现', 26, 'O'],
             ['大', 27, 'O'],
             ['肠', 28, 'O'],
             ['多', 29, '疾病-B'],
             ['发', 30, '疾病-I'],
             ['息', 31, '疾病-I'],
             ['肉', 32, '疾病-E'],
             ['3', 33, '修饰-B'],
             ['月', 34, '修饰-I'],
             ['余', 35, '修饰-E'],
             ['”', 36, 'O'],
             ['入', 37, 'O'],
             ['院', 38, 'O'],
             ['。', 39, 'O'],
             ['查', 40, 'O'],
             ['体', 41, 'O'],
             [':', 42, 'O'],
             ['无', 43, '不确定-B'],
             ['阳', 44, '不确定-I'],
             ['性', 45, '不确定-I'],
             ['体', 46, '不确定-I'],
             ['征', 47, '不确定-E'],
             ['。', 48, 'O']]

def getCITSents(strSents, CITText):
    lenLastSent = 0
    collapse    = 0 # don't need to move 
    CITSents = []
    for strSent in strSents:
        CITSent = []
        for sentTokenIdx, c in enumerate(strSent):
            # sentTokenIdx = txtTokenIdx - lenLastSent - collapse
            txtTokenIdx = sentTokenIdx + lenLastSent + collapse
            cT, _, tT = CITText[txtTokenIdx]
            while c != cT and c != ' ':
                collapse = collapse + 1
                txtTokenIdx = sentTokenIdx + lenLastSent + collapse
                cT, _, tT = CITText[txtTokenIdx]
            CITSent.append([c,sentTokenIdx, tT])
        lenLastSent = lenLastSent + len(strSent)
        CITSents.append(CITSent)
    # CITSents
    # Here we get CITSents  
    return CITSents
       
    
CITSents = getCITSents(strSents, CITText)
CITSents

[[['结', 0, 'O'],
  ['肠', 1, 'O'],
  ['多', 2, '疾病-B'],
  ['发', 3, '疾病-I'],
  ['息', 4, '疾病-I'],
  ['肉', 5, '疾病-E'],
  ['。', 6, 'O']],
 [['患', 0, 'O'],
  ['中', 1, 'O'],
  ['老', 2, 'O'],
  ['年', 3, 'O'],
  ['男', 4, 'O'],
  ['性', 5, 'O'],
  [',', 6, 'O'],
  ['慢', 7, '修饰-B'],
  ['性', 8, '修饰-E'],
  ['病', 9, 'O'],
  ['程', 10, 'O'],
  ['。', 11, 'O']],
 [['因', 0, 'O'],
  ['“', 1, 'O'],
  ['体', 2, 'O'],
  ['检', 3, 'O'],
  ['发', 4, 'O'],
  ['现', 5, 'O'],
  ['大', 6, 'O'],
  ['肠', 7, 'O'],
  ['多', 8, '疾病-B'],
  ['发', 9, '疾病-I'],
  ['息', 10, '疾病-I'],
  ['肉', 11, '疾病-E'],
  ['3', 12, '修饰-B'],
  ['月', 13, '修饰-I'],
  ['余', 14, '修饰-E'],
  ['”', 15, 'O'],
  ['入', 16, 'O'],
  ['院', 17, 'O'],
  ['。', 18, 'O']],
 [['查', 0, 'O'],
  ['体', 1, 'O'],
  [':', 2, 'O'],
  ['无', 3, '不确定-B'],
  ['阳', 4, '不确定-I'],
  ['性', 5, '不确定-I'],
  ['体', 6, '不确定-I'],
  ['征', 7, '不确定-E'],
  ['。', 8, 'O']]]

## getSSET_from_CIT

In [6]:
CITSent = CITSents[0]
print(CITSent)
tag_seq = [i[-1] for i in CITSent]
tag_seq

[['结', 0, 'O'], ['肠', 1, 'O'], ['多', 2, '疾病-B'], ['发', 3, '疾病-I'], ['息', 4, '疾病-I'], ['肉', 5, '疾病-E'], ['。', 6, 'O']]


['O', 'O', '疾病-B', '疾病-I', '疾病-I', '疾病-E', 'O']

In [ ]:
def getSSET_from_CIT(orig_seq = None, tag_seq = None, CIT = None, tag_seq_tagScheme = 'BIO', join_char = ''):
    # orig_seq is sentence without start or end
    # tag_seq may have start or end
        
    tagScheme = tag_seq_tagScheme
    if tagScheme == 'BIOES':
        tag_seq = [i.replace('-S', '-B').replace('-E', '-I') for i in tag_seq]
    elif tagScheme == 'BIOE':
        tag_seq = [i.replace('-E', '-I') for i in tag_seq]
    elif tagScheme == 'BIOS':
        tag_seq = [i.replace('-S', '-B') for i in tag_seq]
    elif tagScheme == 'BIO':
        pass
    else:
        print('The tagScheme', tagScheme, 'is not supported yet...')
    
    if not CIT:
        # use BIO tagScheme
        CIT = list(zip(orig_seq, range(len(orig_seq)), tag_seq))
    taggedCIT = [cit for cit in CIT if cit[2]!= 'O']
    
    startIdx = [idx for idx in range(len(taggedCIT)) if taggedCIT[idx][2][-2:] == '-B']
    startIdx.append(len(taggedCIT))

    entitiesList = []
    for i in range(len(startIdx)-1):
        entityAtom = taggedCIT[startIdx[i]: startIdx[i+1]]
        string = join_char.join([cit[0] for cit in entityAtom])
        start, end = entityAtom[0][1], entityAtom[-1][1] + 1
        tag = entityAtom[0][2].split('-')[0]
        entitiesList.append((string, start, end, tag))
    return entitiesList

# Get strText, strSents, and SSET